In [61]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from scipy import stats

In [62]:
df=pd.read_csv("Pitchfork2.csv",encoding = "ISO-8859-1")

In [63]:
df=df.drop('Unnamed: 0',axis=1)
df=df.drop_duplicates()
df['album']=df['album'].str.split("\r\n",expand=True)[0]

In [64]:
month_scraped=12
day_scraped=23
df['Release Year']=df['years'].str.split("/",expand=True)[0]
df['day']=df['dates'].str.split(" ",expand=True)[1]
df.loc[df['day']=="hrs",'day']=day_scraped
df.loc[df['Release Year'].isnull(),'Release Year']=df.loc[df['Release Year'].isnull(),'dates'].str.split(" ",expand=True)[2]
df['month']=month_scraped
def get_month(name,number):
    df.loc[df['dates'].str.split(" ",expand=True)[0]==name,'month']=number
months=["January","February","March","April","May","June","July","August","September","October","November","December"]
for i in range(12):
    get_month(months[i],i+1)
df['dates']=pd.to_datetime(df['dates'])

In [65]:
def genre_bool(genre_name):
    df[genre_name]=0
    df.loc[df['genre'].str.lower().str.contains(genre_name)==True,genre_name]=1
genre_bool("electronic")
genre_bool("pop")
genre_bool("r&b")
genre_bool("rap")
genre_bool("experimental")
genre_bool("rock")
genre_bool("global")
genre_bool("folk")
genre_bool("country")
genre_bool("jazz")
genre_bool("experimental")

In [77]:
def change_release_year(df,album,artist,year):
    df.loc[(df['artists']==artist) & (df['album']==album),'Release Year']=year
change_release_year(df,'Meat Is Murder','The Smiths',1985)
change_release_year(df,'The Queen Is Dead','The Smiths',1986)
change_release_year(df,'The Smiths','The Smiths',1984)
change_release_year(df,'Strangeways, Here We Come','The Smiths',1987)
change_release_year(df,'Rank','The Smiths',1988)
change_release_year(df,'Hatful of Hollow','The Smiths',1984)
change_release_year(df,"The World Won't Listen",'The Smiths',1987)
change_release_year(df,'Louder Than Bombs','The Smiths',1987)
change_release_year(df,'Disintegration [Deluxe Edition]','The Cure',1989)
change_release_year(df,"Tim","The Replacements",1985)
change_release_year(df,"Pleased to Meet Me","The Replacements",1987)
change_release_year(df,"Don't Tell a Soul","The Replacements",1989)
change_release_year(df,"Sorry Ma, Forgot to Take Out the Trash","The Replacements",1981)
change_release_year(df,"All Shook Down","The Replacements",1990)
change_release_year(df,"Stink","The Replacements",1982)
change_release_year(df,"Hootenanny","The Replacements",1983)
change_release_year(df,"Let It Be","The Replacements",1984)
change_release_year(df,"Spiderland","Slint",1991)
change_release_year(df,"The Disintegration Loops","William Basinski",2002)
change_release_year(df,"On Fire","Galaxie 500",1989)
change_release_year(df,'Laughing Stock',"Talk Talk",1991)
change_release_year(df,'Voodoo','DAngelo',2000)
change_release_year(df,'The Velvet Underground & Nico','The Velvet Underground',1967)
change_release_year(df,"Power, Corruption and Lies",'New Order',1983)
change_release_year(df,"Movement",'New Order',1981)
change_release_year(df,"Low-Life",'New Order',1985)
change_release_year(df,"Brotherhood",'New Order',1986)
change_release_year(df,"Technique (deluxe editions)",'New Order',1989)
change_release_year(df,"Animals",'Pink Floyd',1977)
change_release_year(df,"May 1977: Get Shown the Light","Grateful Dead",1977)
change_release_year(df,"Thelonious Monk: Les Liaisons Dangereuses 1960","Thelonious Monk",1960)

In [67]:
df['Reissue']=0
df.loc[df['album'].str.lower().str.contains("reissue")==True,'Reissue']=1
df.loc[df['album'].str.lower().str.contains("deluxe")==True,'Reissue']=1
df.loc[df['album']=="Cupid Deluxe",'Reissue']=0
df.loc[df['album']=="Moon Deluxe",'Reissue']=0
df.loc[df['album']=="Love Deluxe",'Reissue']=0
df.loc[df['album'].str.lower().str.contains("anniversary")==True,'Reissue']=1
df.loc[df['album']=="Anniversary",'Reissue']=0
df.loc[df['album']=="Protest a Dark Anniversary",'Reissue']=0
replace=df.loc[(df['years'].str.contains("/")==True) & (df['Release Year'].astype(int)>2011) & (df['Release Year'].astype(int)<2015) 
                & (df['years'].str.split("/",expand=True)[1]!="2016"),'years'].str.split("/",expand=True)[1]
df.loc[(df['years'].str.contains("/")==True) & (df['Release Year'].astype(int)>2011) & (df['Release Year'].astype(int)<2015) 
       & (df['years'].str.split("/",expand=True)[1]!="2016"),'Release Year']=replace

df.loc[(df['album'].str.lower().str.contains('legacy')==True) & (df['album'].str.lower().str.contains('tron')==False),'Reissue']=1

In [68]:
df['Release Year']=df['Release Year'].astype(int)
df['month']=df['month'].astype(int)
df['day']=df['day'].astype(int)
df=df.loc[df['Reissue']==0].sort_values(['artists','Release Year','month','day'],ascending=False)
def get_previous_scores(number):
    df['Prev'+str(number)]=df['scores'].shift(-number)
    df.loc[df['artists']!=df['artists'].shift(-number),'Prev'+str(number)]=-1
for i in range(10):
    get_previous_scores(i+1)

In [73]:
df['Wait']=df['Release Year']-df['Release Year'].shift(-1)
df.loc[df['artists']!=df['artists'].shift(-1),'Wait']=-1
df['Label Switch']=0
df.loc[(df['labels']!=df['labels'].shift(-1)) & (df['artists']==df['artists'].shift(-1)),'Label Switch']=1

In [75]:
df['Major']=0
def majorLabel(label):
    df.loc[df['labels'].str.contains(label)==True,'Major']=1
df['MajorIndie']=0
def majorIndieLabel(label):
    df.loc[df['labels'].str.contains(label)==True,'MajorIndie']=1
majorLabel("Warner")
majorLabel("Universal")
majorLabel("Sony")
majorLabel("Columbia")
majorLabel("EMI")
majorLabel("Def Jam")
majorLabel("Roc-A-Fella")
majorLabel("Republic")
majorLabel("Capitol")
majorLabel("GOOD")
majorLabel("Atlantic")
majorLabel("Rhino")
majorLabel("Interscope")
majorLabel("Shady")
majorLabel("Flawless")
majorLabel("Geffen")
majorLabel("RCA")
majorLabel("Epic")
majorLabel("Legacy")
majorLabel("Provident")
majorLabel("Aftermath")
majorLabel("Astralwerks")
majorLabel("Virgin")
majorLabel("Rhino")
majorIndieLabel("XL")
majorIndieLabel("Sub Pop")
majorIndieLabel("Drag City")
majorIndieLabel("Thrill Jockey")
majorIndieLabel("Merge")
majorIndieLabel("Warp")
majorIndieLabel("Domino")
majorIndieLabel("Matador")
majorIndieLabel("Mute")
majorIndieLabel("Jagjaguwar")
majorIndieLabel("Secretly Canadian")
majorIndieLabel("4AD")
majorIndieLabel("Polyvinyl")
majorIndieLabel("Stones Throw")
df['Other']=0
df.loc[(df['Major']==0) & (df['MajorIndie']==0),'Other']=1

In [78]:
df.to_csv("Pitchforkcleaned.csv")